## Using the Workforce Python Module to Automate Workforce for ArcGIS

Workforce for ArcGIS is a mobile solution that uses the power of location-based decision making for better field workforce coordination and teamwork. It is composed of a web app used by project administrators and dispatchers in the office, and a mobile app used by mobile workers on their devices. Organizations using Workforce for ArcGIS get these benefits:

- Everything you need on one device—Mobile workers can easily view and process work assignments, provide updates on work status, and inform others of their location, all from one device.

- Greater agility—Using real-time and location-based information, dispatchers can assign and prioritize fieldwork on the fly and ensure that work is assigned to the right people at the right time.

- Increased productivity—Replace time-consuming and error-prone manual workforce management processes, reduce downtime, and keep projects on schedule.

### Workforce Schema

**Note**: The following was extracted/summarized from [here](https://doc.arcgis.com/en/workforce/android-phone/help/workforce-schema.htm).

A workforce project is composed of four feature layers and four coded value domains with a predefined schema. The name of each feature layer is a combination of a moniker, describing the purpose of the feature layer, appended with the GUID of the Workforce project item. For example, the Workers layer associated with a project with GUID 5dd018fcd88c4d33814cf3da9c44061e would be named workers_5dd018fcd88c4d33814cf3da9c44061e. This guarantees uniqueness of each feature layer.

The four feature layers are as follows:

- **Workers**

  - A point feature layer that contains a record for each mobile worker who is included in the project.
  - Includes information about the mobile worker, including their contact number and job title.
  - The mobile worker's ArcGIS organizational user name is stored in the userId field.
  - The layer tracks who created and last updated each mobile worker.
  - There is a primary key-foreign key (PK-FK) relationship from OBJECTID to Assignments.workerId. Using the OBJECTID value from the Workers layer as the Assignments.workerId field value associates the mobile worker with all their assignments.
  - The layer has the following coded value domain associations:
  - The status field is associated with the Worker_Status coded value domain to track the mobile worker status.

- **Assignments**

  - A point feature layer that contains a record for each assignment.
  - Includes information about the assignment, including its status, location, and description, among others.
  - The layer tracks who created and last updated each assignment.
  - Attachments are enabled on the feature layer.
  - The layer contains foreign keys for some fields, associating values from another layer with this layer:
    - Assignments.workerId to Workers.OBJECTID.
    - Assignments.dispatcherId to Dispatchers.OBJECTID.
    - Assignments.workOrderId can be used as a foreign key to an external system, such as an asset or maintenance management system, by providing values from the other system.
  - The layer has the following coded value domain associations:
    - The status field is associated with the Assign_Status coded value domain to track the assignment status.
    - The priority field is associated with the Priority coded value domain to manage the priority of work assignments.
    - The assignmentType field is associated with the Assign_Type coded value domain to store the assignment types for the project.

- **Dispatchers**
  - A point feature layer that contains a record for each dispatcher within the project.
  - Includes information about the dispatcher, including their name and contact number.
  - The dispatcher's ArcGIS organizational user name is stored in the userId field.
  - The layer tracks who created and last updated each dispatcher.
  - There is a PK-FK relationship from OBJECTID to Assignments.dispatcherId. Using the OBJECTID value from the Dispatchers layer as the Assignments.dispatcherId field value associates the dispatcher with all the assignments they assigned.

- **Location Tracking**
  - A point feature layer that contains a record for each point location logged while location tracking is enabled.
  - The layer tracks who created and last updated each location track.
  
Additionally, a workforce project contains two webmaps:

- **Dispatcher Webmap**
  - This map is what the dispatchers using the back-office web app see
  - It shows the assignments and worker locations
  - Additional layers can be added to this map
  
- **Worker Webmap**
  - This map is what a field worker uses on their iOS or Android device
  - Additional layers can be added to this map
  
When a new project is created via the web application, a new **Group** is created. All of the layers, webmaps, and the project item itself are shared into this group.

When a new project is created via the web application, a new **Folder** is created. All of the layers, webmaps, and the project item itself are shared into this folder.

Finally, there is the actual **Workforce Project** item. This is an item on the portal that stores project meta data in json format.

```json
{
    "workerWebMapId": "<worker-map-id>",
    "dispatcherWebMapId": "<dispatcher-map-id>",
    "dispatchers": {
        "serviceItemId": "<item-id>",
        "url": "<layer-url>"
    },
    "assignments": {
        "serviceItemId": "<item-id>",
        "url": "<layer-url>"
    },
    "workers": {
        "serviceItemId": "<item-id>",
        "url": "<layer-url>"
    },
    "tracks": {
        "serviceItemId": "<item-id>",
        "url": "<layer-url>",
        "enabled": <true | false>,
        "updateInterval": 30
    },
    "version": "1.2.0",
    "groupId": "<group-id>",
    "folderId": "<folder-d>",
    "assignmentIntegrations": [
        {
            "id": "default-navigator",
            "prompt": "Navigate to Assignment",
            "urlTemplate": "arcgis-navigator://?stop=${assignment.latitude},${assignment.longitude}&stopname=${assignment.location}&callback=arcgis-workforce://&callbackprompt=Workforce"
        }
    ]
}
```


### Common tasks that can be accomplished with this module

#### Workers and Dispatchers
- Adding Dispatchers and Workers to a Project
- Deleting Dispatchers and Workers from a Project
- Updating Workers and Dispatchers in a Project
- Searching Workers and Dispatchers in a Project

#### Assignments
- Adding Assignments to a Project
- Deleting Assignments from a Project
- Updating Assignments in a Project
- Assigning Assignments in a Project
- Searching Assignments in a Project
- Adding/Removing/Downloading Attachments

#### Tracks
- Searching Tracks (for analysis)

### Getting Started

A user must be authenticated with a GIS in order to fetch a Project. The workforce functionality is available in `arcgis.apps.workforce`

In [1]:
from arcgis.gis import GIS
from arcgis.apps import workforce

gis = GIS('https://arcgis.com', 'workforce_scripts')

Enter password: ········


#### Workforce Project
A project is created using its corresponding item.

In [2]:
item = gis.content.get("29eec3c45982458d876c5e1f2f32333d")
project = workforce.Project(item)

#### Assignments
Assignments are accessed by using the assignment manager object. Assignments can be added, updated, or deleted. Additionally, attachments can be added, deleted, or downloaded by using the assignment attachment manager object.

In [3]:
# Search all assignments
assignments = project.assignments.search()

# View first assignment
assignment = assignments[0]
print(f"Status: {assignment.status}")
print(f"Description: {assignment.description}")
print(f"Priority: {assignment.priority}")
print(f"Assigned To: {assignment.worker.name}")
print(f"Type: {assignment.assignment_type}")

# Update the description of the assignment
assignment.update(description="You need to do an inspection here")
print("--------------------")
print(f"Updated Description: {project.assignments.search()[0].description}")

# Download the assignment using the AssignmentAttachmentManager
assignment.attachments.download()

Status: assigned
Description: Do some work at the ESRI R&D Center
Priority: medium
Assigned To: Aaron Pulver
Type: Inspection
--------------------
Updated Description: You need to do an inspection here


['/Users/aaro8157/PycharmProjects/arcgis-python-api/samples/02_power_users_developers/esri_logo1.png']

#### Assignment Types
Assignment types are accessed by using an assignment type manager. Assignment types can be added, updated, or deleted.

In [4]:
# List all assignment types
assignment_types = project.assignment_types.search()
for at in assignment_types:
    print(f"Type: {at.name}")
    
# Add a new assignment type
project.assignment_types.add(name="Repair")

# Confirm that it was added
print("--------------------")
assignment_types = project.assignment_types.search()
for at in assignment_types:
    print(f"Type: {at.name}")

Type: Inspection
Type: Removal
--------------------
Type: Inspection
Type: Removal
Type: Repair


#### Workers
Workers are accessed by using a worker manager object. Workers can be added, updated, or deleted.

In [5]:
# Search all workers and print details about first worker 
workers = project.workers.search()
worker = workers[0]
print(f"Name: {worker.name}")
print(f"Number: {worker.contact_number}")
    
# Update the workers contact number
worker.update(contact_number="123-456-7890")
print("--------------------")
print(f"Number: {project.workers.search()[0].contact_number}")

Name: Aaron Pulver
Number: None
--------------------
Number: 123-456-7890


In [6]:
# Add a new worker
project.workers.add(name="Demo User", 
                    user_id="demouser_nitro", 
                    contact_number="123-987-4560")

<Worker 34>

#### Dispatchers
Dispatchers are accessed by using a dispatcher manager object. Dispatchers can be added, updated, or deleted.

In [7]:
# Search for all dispatchers and print details about first dispatcher
dispatchers = project.dispatchers.search()
dispatcher = dispatchers[0]
print(f"Name: {dispatcher.name}")
print(f"Number: {dispatcher.contact_number}")
    
# Update the dispatchers contact number
dispatcher.update(contact_number="123-456-7890")
print("--------------------")
print(f"Number: {project.dispatchers.search()[0].contact_number}")

Name: workforce scripts
Number: 987-654-3210
--------------------
Number: 123-456-7890


#### Webmaps
The dispatcher and worker webmaps can be accessed as shown in the following code snippet. By using the mapping module, additional layers could be added to either map.

In [8]:
# Worker webmap
project.worker_webmap

In [9]:
# Dispatcher webmap
project.dispatcher_webmap

### Putting it all together
In the following snippet, a new assignment will be created at the ESRI campus. Assignments (as well as workers, assignment types, dispatchers, and tracks) are all validated prior to upload. The ensures the integrity of the workforce project.

In [10]:
# Add a new assignment and assign it to demouser
from datetime import datetime
demouser = project.workers.get(user_id='demouser_nitro')
dispatcher = project.dispatchers.get(user_id='workforce_scripts')
repair = project.assignment_types.get(name="Repair")

# Use the geocoder to find the location of ESRI
from arcgis.geocoding import geocode
geometry = geocode("ESRI, Redlands, CA", out_sr=102100)[0]['location']

# Add a new assignment
project.assignments.add(assignment_type=repair,
                       status="assigned",
                       assigned_date=datetime.now(),
                       worker=demouser,
                       dispatcher=dispatcher,
                       location="ESRI, Redlands, CA",
                       geometry=geometry)

<Assignment 66>

### Reset the Demo Project
The following code resets the project to the intial state for this demo. It also highlights how batch methods can be used to add/update/delete many workforce items at once. This is useful when processing large amounts of assignments, workers, or dispatchers. In general, batch methods make fewer calls to the backend server thus reducing the overall time of the script.

In [11]:
# Use batch functions to process multiple items at a time
project.assignments.batch_delete(project.assignments.search(where='assignmentType=3'))
project.assignment_types.batch_delete([project.assignment_types.get(name="Repair")])
project.workers.batch_delete([project.workers.get(user_id="demouser_nitro")])

# Reset the assignment description
a = project.assignments.get(object_id=1)
a.update(description="Do some work at the ESRI R&D Center")

# Reset the worker using batch update
w1 = project.workers.get(object_id=1)
w1.contact_number = None
project.workers.batch_update([w1])

# Reset the dispatcher using batch update
d1 = project.dispatchers.get(object_id=1)
d1.contact_number = "987-654-3210"
project.dispatchers.batch_update([d1])

[<Dispatcher 1>]